In [1]:
import torch
from train import *
from data.utils import *
from continuum import ClassIncremental
from continuum.tasks import split_train_val
DEVICE = torch.device("cpu")

In [ ]:
# trainset_cifar, testset_cifar = load_data(dataset='cifar')
# trainloader_cifar = torch.utils.data.DataLoader(dataset=trainset_cifar, batch_size=1, shuffle=True)
# testloader_cifar  = torch.utils.data.DataLoader(dataset=testset_cifar,  batch_size=1, shuffle=False)

trainset_mnist, testset_mnist = load_data(dataset='mnist')
trainloader_mnist = torch.utils.data.DataLoader(dataset=trainset_mnist, batch_size=1, shuffle=True)
testloader_mnist  = torch.utils.data.DataLoader(dataset=testset_mnist,  batch_size=1, shuffle=False)

params = {
    'lr': 0.001,
    'dp': True,
    'C': .02,
    'sigma': .1
    'increment': 5
}


for learning in ['central', 'single', 'multi']:
    if learning == 'central':
        trainloader = torch.utils.data.DataLoader(
            dataset=trainset_mnist, batch_size=params['batch_size'], shuffle=True, drop_last=True)

        testloader = torch.utils.data.DataLoader(
            dataset=testset_mnist, batch_size=params['batch_size'], shuffle=False, drop_last=True)
        model = load_model(dataset='mnist')
        train_results = train( model=model, trainloader=trainloader,
              device=DEVICE, opt_params=params)
        test_results = test(model, testloader_mnist, device=DEVICE)

    else:
        # Incremental Learning(Changing params will change how many classes at a time to learn
        scenario = ClassIncremental(trainset_mnist, increment=(1 if learning == 'single' else params['increment']))
        print(f"Number of classes: {scenario.nb_classes}.")
        print(f"Number of tasks: {scenario.nb_tasks}.")
        for task_id, train_taskset in enumerate(scenario):
            train_taskset, val_taskset = split_train_val(train_taskset, val_split=0)
            trainloader = torch.utils.data.DataLoader(dataset=train_taskset, batch_size=params['batch_size'],
                                                      shuffle=True, drop_last=True)
            model = load_model(dataset='mnist')
            train_results = train( model=model, trainloader=trainloader,
                   device=DEVICE, opt_params=params)
            test_results = test(model, testloader_mnist, device=DEVICE)



